In [3]:
import sys
import os
sys.path.append("./../bin/Release")
sys.path.append("./../lib/Release")
sys.path.append("./../")
import numpy as np
import time
from scipy.optimize import differential_evolution, minimize
import matplotlib.pyplot as plt
import pandas as pd
from pyminion.test import MWUT
from scipy.stats import rankdata

In [4]:
goptimum_cec22 = np.array([300, 400, 600, 800, 900, 1800, 2000, 2200, 2300,  2400,2600,2700])
goptimum_cec20 = np.array([100, 1100,700, 1900, 1700, 1600, 2100, 2200, 2400, 2500])
goptimum_cec17 = np.array([100*i for i in range(1, 31)])
goptimum_cec14= goptimum_cec17
yearToMin = {2017:goptimum_cec17, 2014 : goptimum_cec14, 2020 : goptimum_cec20, 2022:goptimum_cec22}
yearToNfuncs = {2017:30, 2014:30, 2020:10, 2022:12}
resFolder = "./results/"

# Scoring the Performance of DE Variants on Multiple CEC Problems (Higher is better. Maximum score =100)

### CEC 2014, 2017, 2019, 2020, 2022

In [ ]:
algos = ["LSHADE", "j2020", "NLSHADE_RSP", "LSRTDE", "ARRDE",  ]
year=2022

multiplier = 10000
dimToYear = {
    2017 :  {10 : 10*multiplier, 30:30*multiplier, 50:50*multiplier, 100:100*multiplier}, 
    #2017 :  {10 : 10*multiplier, 30:30*multiplier, 50:50*multiplier}, 
    2014 :  {10 : 10*multiplier, 30:30*multiplier, 50:50*multiplier, 100:100*multiplier }, 
    #2020 : {5 : 5*multiplier, 10:10*multiplier, 15:15*multiplier, 20:20*multiplier },
    2020 : {5 : 50000, 10:1000000, 15:3000000, 20:10000000 },
    #2022 : { 10:10*multiplier,  20:20*multiplier },
    2022 : { 10:200000,  20:1000000 }
}
dimMaxEvals = dimToYear[year]

matrices = {}
for algo in algos : 
    matList = []
    for dim, maxevals in dimMaxEvals.items() : 
        filename = resFolder+"results_"+str(year)+"_"+ algo +"_"+str(dim) +"_"+str(maxevals)+".txt" 
        matList.append(
            np.loadtxt(filename, delimiter='\t')
        )
    matrices[algo] = matList

def calcSE(year, userel=True) : 
    ret = {}
    for algo in algos :
        se = 0.0 
        coeff=0.1
        Nfuncs = yearToNfuncs[year]
        globMin = yearToMin[year]
        for arr in matrices[algo] : 
            ae = np.mean(np.abs(np.array(arr)-globMin), axis=0) 
            if userel : ae = np.mean(np.abs(np.array(arr)-globMin)/globMin, axis=0) 
            #print(ae)
            for ifunc in range(Nfuncs) : 
                if year==2017 and ifunc==1 : continue
                #if year==2017 and (ifunc in range( 20)) : continue
                se += coeff*ae[ifunc]
            coeff=coeff+0.1
        ret[algo] = se
    return ret
            
def calcRank(arrDict, ideal, mode="mean") : 
    #here arrlist is [[,,,], [,...]]
    experiments = []
    for algo, arr in arrDict.items() : 
        if year == 2020 : arr = arr[:30]
        experiments.append( np.abs(np.array(arr)-ideal) )
    try : 
        experiments = np.array(experiments)
    except : 
        for e in experiments : print(len(e))
        print(ideal, experiments)
        raise Exception()
    
    if mode=="ranksum" : 
        flattened_data = experiments.flatten()
        ranks = rankdata(flattened_data)
        ranked_experiments = ranks.reshape(experiments.shape)
        total_ranks = np.sum(ranked_experiments, axis=1)
    elif mode == "mean" : 
        total_ranks = np.mean(experiments, axis=1)
    elif mode =="median" :
        total_ranks = np.median(experiments, axis=1)
    else : raise Exception("Uknown mode.")
    
    final_rank = rankdata(total_ranks)
    return dict(zip(arrDict.keys(), final_rank))

def calcSR(year) : 
    Nfuncs = yearToNfuncs[year]
    minimum = yearToMin[year]
    ret = {}
    for algo in algos :
        sr = 0.0 
        coeff=0.1
        Nfuncs = yearToNfuncs[year]
        for idim, arr in enumerate(matrices[algo]) : 
            arrde_rs = []
            lshade_rs = []
            for ifunc in range(Nfuncs) : 
                if year==2017 and ifunc==1 : continue
                #if year==2017 and (ifunc in range( 20)) : continue
                arrDict= {}
                for algo_ in algos : 
                    arrDict[algo_] = matrices[algo_][idim][:, ifunc]
                rankDict = calcRank(arrDict, minimum[ifunc])
                if algo=="LSHADE" : lshade_rs.append(coeff*rankDict[algo])
                if algo=="ARRDE" : arrde_rs.append(coeff*rankDict[algo])

                sr += coeff*rankDict[algo]
            coeff=coeff+0.1
        ret[algo] = sr
    return ret

    
SR = calcSR(year)
SE = calcSE(year, False)
print("SR " ,  SR)
SEmin = np.min(list(SE.values()))
SRmin = np.min(list(SR.values()))
score1= {algo:50.0 * SEmin/SE[algo]  for algo in algos}
score2= {algo:50.0 * SRmin/SR[algo]  for algo in algos}
final = {algo:score1[algo]+score2[algo] for algo in algos}
print("Score1 : ", score1)
print("Score2 : ", score2)
final

word =rf"$S_1$ & " 
for i, (k, v) in enumerate(score1.items()) : 
    word+= f"{v:0.1f}"
    if i!=len(score1)-1 : word+= " & "
    else : word+=rf"\\"
print (word)

word =rf"$S_2$ & " 
for i, (k, v) in enumerate(score2.items()) : 
    word+= f"{v:0.1f}"
    if i!=len(score1)-1 : word+= " & "
    else : word+=rf"\\"
print (word)

word =rf"$S$ & " 
for i, (k, v) in enumerate(final.items()) : 
    word+= f"{v:0.1f}"
    if i!=len(score1)-1 : word+= " & "
    else : word+=rf"\\"
print (word)

In [ ]:
algos = ["LSHADE", "LSRTDE", "ARRDE", "NLSHADE_RSP", "j2020"]
year=2022

multiplierList = [500, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
score1List = []
score2List = []
scoreTotList = []
for multiplier in multiplierList :
    dimToYear = {
        2017 :  {10 : 10*multiplier, 30:30*multiplier, 50:50*multiplier, 100:100*multiplier}, 
        #2017 :  {10 : 10*multiplier, 30:30*multiplier, 50:50*multiplier}, 
        2014 :  {10 : 10*multiplier, 30:30*multiplier, 50:50*multiplier, 100:100*multiplier }, 
        2020 : {5 : 5*multiplier, 10:10*multiplier, 15:15*multiplier, 20:20*multiplier },
        #2020 : {5 : 50000, 10:1000000, 15:3000000, 20:10000000 },
        2022 : { 10:10*multiplier,  20:20*multiplier },
        #2022 : { 10:200000,  20:1000000 }
    }
    if (multiplier ==0) : 
        dimToYear [2020] ={5 : 50000, 10:1000000, 15:3000000, 20:10000000 }
        dimToYear [2022] = { 10:200000,  20:1000000 }
        
    dimMaxEvals = dimToYear[year]

    matrices = {}
    for algo in algos : 
        matList = []
        for dim, maxevals in dimMaxEvals.items() : 
            filename = resFolder+"results_"+str(year)+"_"+ algo +"_"+str(dim) +"_"+str(maxevals)+".txt" 
            matList.append(
                np.loadtxt(filename, delimiter='\t')
            )
        matrices[algo] = matList

    def calcSE(year) : 
        ret = {}
        for algo in algos :
            se = 0.0 
            coeff=0.1
            Nfuncs = yearToNfuncs[year]
            globMin = yearToMin[year]
            for arr in matrices[algo] : 
                ae = np.mean(np.abs(np.array(arr)-globMin), axis=0) 
                #print(ae)
                for ifunc in range(Nfuncs) : 
                    if year==2017 and ifunc==1 : continue
                    se += coeff*ae[ifunc]
                coeff=coeff+0.1
            ret[algo] = se
        return ret
                
    def calcRank(arrDict, ideal, mode="mean") : 
        #here arrlist is [[,,,], [,...]]
        experiments = []
        for algo, arr in arrDict.items() : 
            if year == 2020 : arr = arr[:30]
            experiments.append( np.abs(np.array(arr)-ideal) )
        try : 
            experiments = np.array(experiments)
        except : 
            for e in experiments : print(len(e))
            print(ideal, experiments)
            raise Exception()
        
        if mode=="ranksum" : 
            flattened_data = experiments.flatten()
            ranks = rankdata(flattened_data)
            ranked_experiments = ranks.reshape(experiments.shape)
            total_ranks = np.sum(ranked_experiments, axis=1)
        elif mode == "mean" : 
            total_ranks = np.mean(experiments, axis=1)
        elif mode =="median" :
            total_ranks = np.median(experiments, axis=1)
        else : raise Exception("Uknown mode.")
        
        final_rank = rankdata(total_ranks)
        return dict(zip(arrDict.keys(), final_rank))

    def calcSR(year) : 
        Nfuncs = yearToNfuncs[year]
        minimum = yearToMin[year]
        ret = {}
        for algo in algos :
            sr = 0.0 
            coeff=0.1
            Nfuncs = yearToNfuncs[year]
            for idim, arr in enumerate(matrices[algo]) : 
                for ifunc in range(Nfuncs) : 
                    if year==2017 and ifunc==1 : continue
                    arrDict= {}
                    for algo_ in algos : 
                        arrDict[algo_] = matrices[algo_][idim][:, ifunc]
                    rankDict = calcRank(arrDict, minimum[ifunc])
                    sr += coeff*rankDict[algo]
                coeff=coeff+0.1
            ret[algo] = sr
        return ret
    

    SR = calcSR(year)
    SE = calcSE(year)
    SEmin = np.min(list(SE.values()))
    SRmin = np.min(list(SR.values()))
    score1= {algo:50.0 * SEmin/SE[algo]  for algo in algos}
    score2= {algo:50.0 * SRmin/SR[algo]  for algo in algos}
    final = {algo:score1[algo]+score2[algo] for algo in algos}
    score1List.append(score1)
    score2List.append(score2)
    scoreTotList.append(final)

def plot(resList, score="Score",year=2017, showLegend=True) : 
    algoRes = {algo : [res[algo] for res in resList] for algo in algos}
    xlist = multiplierList
    #xlist = [500, 1000, 5000, 10000]
    plt.figure(figsize=(6,4))
    for algo, res in algoRes.items() : 
        if algo == "NLSHADE_RSP": algo ="NL-SHADE-RSP"
        #print(res)
        plt.plot(xlist, res, label=algo)
        plt.scatter(xlist, res)
    if showLegend : plt.legend() 
    #plt.xscale("log")
    plt.xlabel(r"$N_{max}/D$")
    plt.ylabel(score)
    plt.savefig(score.replace("$", "")+"_"+str(year)+".pdf", bbox_inches="tight")

plot(score1List, r"$S_1$-score", year)
plot(score2List, r"$S_2$-score", year)
plot(scoreTotList, r"$S$-score", year)

### CEC 2011 and Curve Fitting Problem

In [ ]:
algos = ["LSHADE",  "j2020", "NLSHADE_RSP", "LSRTDE", "ARRDE"]
matrices = {}
for algo in algos : 
    filename = resFolder+ algo + "_cec2011_31_full_3"+".txt" 
    matrices[algo] = np.loadtxt(filename)
minimum = []
for ifunc in range(20) : 
    arr = np.array([])
    for algo in algos :
        newarr = matrices[algo][:, ifunc]
        arr = np.concatenate((arr, newarr))
    minimum.append(np.min(arr))

print("Minimum : ", minimum)
def calcSE( relative = True) : 
    ret = {}
    for algo in algos :
        se = 0.0 
        for ifunc in range(0, 20) : 
            if not relative :
                if minimum[ifunc]!=0.0 :
                    scale = 100.0/np.abs(minimum[ifunc])
                else : 
                    scale =0.001
                se +=  scale * np.mean(np.abs(matrices[algo][:, ifunc]-minimum[ifunc]))
            else : 
                if minimum[ifunc]==0.0 : se +=  np.mean(matrices[algo][:, ifunc])
                else : se +=  np.mean( (matrices[algo][:, ifunc]-minimum[ifunc])/np.abs(minimum[ifunc]))
        ret[algo] = se
    return ret
            
def calcRank(arrDict, ideal, mode="mean") : 
    #here arrlist is [[,,,], [,...]]
    experiments = []
    for algo, arr in arrDict.items() : 
        experiments.append( np.abs(np.array(arr)-ideal) )
    try : 
        experiments = np.array(experiments)
    except : 
        for e in experiments : print(len(e))
        print(ideal, experiments)
        raise Exception()
    if mode=="ranksum" :
        flattened_data = experiments.flatten()
        ranks = rankdata(flattened_data)
        ranked_experiments = ranks.reshape(experiments.shape)
        total_ranks = np.sum(ranked_experiments, axis=1)
    elif mode == "mean" : 
        total_ranks = np.mean(experiments, axis=1)
    elif mode =="median" :
        total_ranks = np.median(experiments, axis=1)
    else : raise Exception("Uknown mode.")
    final_rank = rankdata(total_ranks)
    return dict(zip(arrDict.keys(), final_rank))

def calcSR() : 
    ret = {}
    arr = []
    for algo in algos :
        sr = 0.0 
        for ifunc in range(0, 20) : 
            arrDict= {}
            for algo_ in algos : 
                arrDict[algo_] = matrices[algo_][:, ifunc]
            rankDict = calcRank(arrDict, minimum[ifunc])
            sr += rankDict[algo]
        ret[algo] = sr
    print(np.sum(arr, axis=0))
    return ret

    #
SR = calcSR()
SE = calcSE(relative=True)

print(SE)
SEmin = np.min(list(SE.values()))
SRmin = np.min(list(SR.values()))
score1= {algo:50.0 * SEmin/SE[algo]  for algo in algos}
score2= {algo:50.0 * SRmin/SR[algo]  for algo in algos}
final = {algo:score1[algo]+score2[algo] for algo in algos}


print("Score1 : ", score1)
print("Score2 : ", score2)
print("S : ", final)

word =rf"$S_1$ & " 
for i, (k, v) in enumerate(score1.items()) : 
    word+= f"{v:0.1f}"
    if i!=len(score1)-1 : word+= " & "
    else : word+=rf"\\"
print (word)

word =rf"$S_2$ & " 
for i, (k, v) in enumerate(score2.items()) : 
    word+= f"{v:0.1f}"
    if i!=len(score1)-1 : word+= " & "
    else : word+=rf"\\"
print (word)

word =rf"$S$ & " 
for i, (k, v) in enumerate(final.items()) : 
    word+= f"{v:0.1f}"
    if i!=len(score1)-1 : word+= " & "
    else : word+=rf"\\"
print (word)


# CEC 2011

In [ ]:
suff = "_3"
filename = resFolder+"NLSHADE_RSP_cec2011_31_full"+suff+".txt"
data = np.loadtxt(filename)
filename = resFolder+"ARRDE_cec2011_31_full"+suff+".txt"
data1 = np.loadtxt(filename)
filename = resFolder+"j2020_cec2011_31_full"+suff+".txt"
data2 = np.loadtxt(filename)
filename = resFolder+"LSHADE_cec2011_31_full"+suff+".txt"
data3 = np.loadtxt(filename)
filename = resFolder+"LSRTDE_cec2011_31_full"+suff+".txt"
data5 = np.loadtxt(filename)

algos = ["LSHADE", "LSRTDE", "ARRDE", "NLSHADE_RSP", "j2020"]
matrices = {}
for algo in algos : 
    filename = resFolder+ algo + "_cec2011_31_full"+suff+".txt" 
    matrices[algo] = np.loadtxt(filename)
minimum = []
for ifunc in range(20) : 
    arr = np.array([])
    for algo in algos :
        newarr = matrices[algo][:, ifunc]
        arr = np.concatenate((arr, newarr))
    minimum.append(np.min(arr))

print("Minimum : ", minimum)

denom = np.abs(minimum)
for i in range(len(denom)) : 
    if denom[i] == 0.0 : denom[i]=1
denom =1.0
minimum = 0.0
df1 = pd.DataFrame({
    "LSHADE" : np.mean((data3-minimum)/denom, axis=0), 
    "j2020" : np.mean((data2-minimum)/denom, axis=0), 
    "NLSHADE_RSP" : np.mean((data-minimum)/denom, axis=0),   
    "LSRTDE" : np.mean((data5-minimum)/denom, axis=0),
    "ARRDE" : np.mean((data1-minimum)/denom, axis=0),  
    })

df2 = pd.DataFrame({
    "LSHADE" : np.std((data3-minimum)/denom, axis=0), 
    "j2020" : np.std((data2-minimum)/denom, axis=0), 
    "NLSHADE_RSP" : np.std((data-minimum)/denom, axis=0),     
    "LSRTDE" : np.std((data5-minimum)/denom, axis=0), 
    "ARRDE" : np.std((data1-minimum)/denom, axis=0), 
    })

display(df1)
display(df2)
#for c in df1.columns : print(c, np.mean(df1[c]))


print("------------")
signMap = {
    0 : "=", 1:"+", -1:"-"
}
Nprobs=20
df = pd.DataFrame({
    "NLSHADE_RSP" : [ signMap[MWUT(data1[:, i], data[:,  i])] for i in range(Nprobs )], 
    "j2020" : [ signMap[MWUT(data1[:, i], data2[:,  i])] for i in range(Nprobs )], 
    "LSHADE" : [ signMap[MWUT(data1[:, i], data3[:,  i])] for i in range(Nprobs )], 
    "LSRTDE" : [ signMap[MWUT(data1[:, i], data5[:,  i])] for i in range(Nprobs )], 
    })

display(df)

dimensions = [
    6, 30, 30, 20, 7, 126, 12, 120, 216, 6, 
    13, 15, 40, 140, 96, 96, 96, 96, 26, 22
]

nzp = {
    "LSHADE" : [0, 0, 0], "j2020" : [0, 0, 0], "NLSHADE_RSP": [0, 0, 0], "LSRTDE" : [0, 0,0]
}

algolist = ["LSHADE", "j2020", "NLSHADE_RSP", "LSRTDE", "ARRDE"]
for i, row in df.iterrows() :
    probnum = i+1 
    if i>=2 : probnum = i+3
    
    indexMax = np.argmax(df1.iloc[i].to_numpy())
    indexMin = np.argmin(df1.iloc[i].to_numpy())

    word = f"{probnum} & {dimensions[i]} & "
    if indexMin ==indexMax : 
        for j, algo in enumerate(algolist) : 
            word += r"\textbf{" 
            word += f"{df1[algo][i]:0.2e} $\pm$ {df2[algo][i]:0.2e} "
            if algo !="ARRDE" :word += f"({row[algo]})"
            word += r"}"

            if j!= (len(algolist)-1) :  word+=" & "
            else : word += r" \\ "

        print(word)
        for rn in ["LSHADE", "j2020", "NLSHADE_RSP", "LSRTDE"] : 
            it = row[rn] 
            if it=="-" : nzp[rn][0]+=1
            elif it == "=" : nzp[rn][1]+=1
            elif it =="+" : nzp[rn][2]+=1

        continue 

    for j, algo in enumerate(algolist) : 
        if j==indexMax : 
            word += r"\color{red} {\textbf{" 
            word += f"{df1[algo][i]:0.2e} $\pm$ {df2[algo][i]:0.2e} "
            if algo !="ARRDE" :word += f"({row[algo]})"
            word += r"}}"

        elif j== indexMin : 
            word += r"\textbf{" 
            word += f"{df1[algo][i]:0.2e} $\pm$ {df2[algo][i]:0.2e} "
            if algo !="ARRDE" :word += f"({row[algo]})"
            word += r"}"
        else : 
            word += f"{df1[algo][i]:0.2e} $\pm$ {df2[algo][i]:0.2e} "  
            if algo !="ARRDE" :word += f"({row[algo]})"

        if j!= (len(algolist)-1) :  word+=" & "
        else : word += r" \\ "

    print(word)
    #print(rf"{probnum} & {df1["LSHADE"][i]:0.2e} $\pm$ {df2["LSHADE"][i]:0.2e} ({row["LSHADE"]}) & {df1["j2020"][i]:0.2e} $\pm$ {df2["j2020"][i]:0.2e} ({row["j2020"]})& {df1["NLSHADE_RSP"][i]:0.2e} $\pm$ {df2["NLSHADE_RSP"][i]:0.2e} ({row["NLSHADE_RSP"]}) & {df1["LSRTDE"][i]:0.2e} $\pm$ {df2["LSRTDE"][i]:0.2e} ({row["LSRTDE"]}) &{df1["ARRDE"][i]:0.2e} $\pm$ {df2["ARRDE"][i]:0.2e}  \\")

    for rn in ["LSHADE", "j2020", "NLSHADE_RSP", "LSRTDE"] : 
        it = row[rn] 
        if it=="-" : nzp[rn][0]+=1
        elif it == "=" : nzp[rn][1]+=1
        elif it =="+" : nzp[rn][2]+=1

print (nzp)

In [ ]:
df_data["LSHADE"].to_numpy()

In [ ]:
df_data

In [ ]:
df_data = pd.read_csv(resFolder+"/df_NN_30.csv")

algolist = ["LSHADE", "j2020", "NLSHADE_RSP", "LSRTDE", "ARRDE"]
algolist_wo = ["LSHADE", "j2020", "NLSHADE_RSP", "LSRTDE"]

df1 = pd.DataFrame({
    algo : [np.mean(df_data[algo].to_numpy())] for algo in algolist
    })

df2 = pd.DataFrame({
  algo : [np.std(df_data[algo].to_numpy())] for algo in algolist
    })

display(df1)
display(df2)
#for c in df1.columns : print(c, np.mean(df1[c]))


signMap = {
    0 : "=", 1:"+", -1:"-"
}

df = pd.DataFrame({
    algo : [ signMap[MWUT(df_data["ARRDE"].to_numpy(), df_data[algo].to_numpy())]] for algo in algolist_wo
})
display(df)


nzp = {
    "LSHADE" : [0, 0, 0], "j2020" : [0, 0, 0], "NLSHADE_RSP": [0, 0, 0], "LSRTDE" : [0, 0,0]
}


for i, row in df.iterrows() :
   
    indexMax = np.argmax(df1.iloc[i].to_numpy())
    indexMin = np.argmin(df1.iloc[i].to_numpy())

    word = f" & "
    if indexMin ==indexMax : 
        for j, algo in enumerate(algolist) : 
            word += r"\textbf{" 
            word += f"{df1[algo][i]:0.2e} $\pm$ {df2[algo][i]:0.2e} "
            if algo !="ARRDE" :word += f"({row[algo]})"
            word += r"}"

            if j!= (len(algolist)-1) :  word+=" & "
            else : word += r" \\ "

        print(word)
        for rn in ["LSHADE", "j2020", "NLSHADE_RSP", "LSRTDE"] : 
            it = row[rn] 
            if it=="-" : nzp[rn][0]+=1
            elif it == "=" : nzp[rn][1]+=1
            elif it =="+" : nzp[rn][2]+=1

        continue 

    for j, algo in enumerate(algolist) : 
        if j==indexMax : 
            word += r"\color{red} {\textbf{" 
            word += f"{df1[algo][i]:0.2e} $\pm$ {df2[algo][i]:0.2e} "
            if algo !="ARRDE" :word += f"({row[algo]})"
            word += r"}}"

        elif j== indexMin : 
            word += r"\textbf{" 
            word += f"{df1[algo][i]:0.2e} $\pm$ {df2[algo][i]:0.2e} "
            if algo !="ARRDE" :word += f"({row[algo]})"
            word += r"}"
        else : 
            word += f"{df1[algo][i]:0.2e} $\pm$ {df2[algo][i]:0.2e} "  
            if algo !="ARRDE" :word += f"({row[algo]})"

        if j!= (len(algolist)-1) :  word+=" & "
        else : word += r" \\ "

    print(word)
    #print(rf"{probnum} & {df1["LSHADE"][i]:0.2e} $\pm$ {df2["LSHADE"][i]:0.2e} ({row["LSHADE"]}) & {df1["j2020"][i]:0.2e} $\pm$ {df2["j2020"][i]:0.2e} ({row["j2020"]})& {df1["NLSHADE_RSP"][i]:0.2e} $\pm$ {df2["NLSHADE_RSP"][i]:0.2e} ({row["NLSHADE_RSP"]}) & {df1["LSRTDE"][i]:0.2e} $\pm$ {df2["LSRTDE"][i]:0.2e} ({row["LSRTDE"]}) &{df1["ARRDE"][i]:0.2e} $\pm$ {df2["ARRDE"][i]:0.2e}  \\")

    for rn in ["LSHADE", "j2020", "NLSHADE_RSP", "LSRTDE"] : 
        it = row[rn] 
        if it=="-" : nzp[rn][0]+=1
        elif it == "=" : nzp[rn][1]+=1
        elif it =="+" : nzp[rn][2]+=1

print (nzp)

# CEC 2017, 2020, 2022

In [ ]:
year =2017
dim=100
if year ==2017 :
    maxevals=dim*10000
elif year==2020 : 
    if dim==5 : maxevals=50000
    elif dim ==10 : maxevals=1000000
    elif dim==15 : maxevals=3000000 
    else : maxevals=10000000 
elif year == 2022 : 
    if dim==10 : maxevals=200000 
    else : maxevals = 1000000

filename = resFolder+"results_"+str(year)+"_NLSHADE_RSP_"+str(dim)+ "_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_"+str(year)+"_ARRDE_"+str(dim)+ "_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_"+str(year)+"_j2020_"+str(dim)+"_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_"+str(year)+"_LSHADE_"+str(dim)+"_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_"+str(year)+"_LSRTDE_"+str(dim)+"_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')

minimum = 0.0 
if year==2017 : minimum = goptimum_cec17 
elif year ==2020 : minimum = goptimum_cec20 
elif year == 2022 : minimum = goptimum_cec22
df1 = pd.DataFrame({
    "LSHADE" : np.mean(np.abs(data3-minimum), axis=0), 
    "j2020" : np.mean(np.abs(data2-minimum), axis=0), 
    "NLSHADE_RSP" : np.mean(np.abs(data-minimum), axis=0),     
    "LSRTDE" : np.mean(np.abs(data5-minimum), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-minimum), axis=0), 
    })

df2 = pd.DataFrame({
    "LSHADE" : np.std(np.abs(data3-minimum), axis=0), 
    "j2020" : np.std(np.abs(data2-minimum), axis=0), 
    "NLSHADE_RSP" : np.std(np.abs(data-minimum), axis=0), 
    "LSRTDE" : np.std(np.abs(data5-minimum), axis=0), 
    "ARRDE" : np.std(np.abs(data1-minimum), axis=0), 
    })

Nprobs = df1.shape[0]
print("------------")
signMap = {
    0 : "=", 1:"+", -1:"-"
}
df = pd.DataFrame({
    "NLSHADE_RSP" : [ signMap[MWUT(data1[:, i], data[:,  i])] for i in range(Nprobs )], 
    "j2020" : [ signMap[MWUT(data1[:, i], data2[:,  i])] for i in range(Nprobs )], 
    "LSHADE" : [ signMap[MWUT(data1[:, i], data3[:,  i])] for i in range(Nprobs )], 
    "LSRTDE" : [ signMap[MWUT(data1[:, i], data5[:,  i])] for i in range(Nprobs )], 
    })
#for c in df.columns : print(c, np.mean(df[c]))


nzp = {
    "LSHADE" : [0, 0, 0], "j2020" : [0, 0, 0], "NLSHADE_RSP": [0, 0, 0], "LSRTDE" : [0, 0,0]
}
algolist = ["LSHADE", "j2020", "NLSHADE_RSP", "LSRTDE", "ARRDE"]
for i, row in df.iterrows() :
    if year==2017 and i==1 : continue 
    #print(row)
    probnum = i+1 
    
    indexMax = np.argmax(df1.iloc[i].to_numpy())
    indexMin = np.argmin(df1.iloc[i].to_numpy())

    word = f"{probnum} & "
    if indexMin ==indexMax : 
        for j, algo in enumerate(algolist) : 
            word += r"\textbf{" 
            word += f"{df1[algo][i]:0.2e} $\pm$ {df2[algo][i]:0.2e} "
            if algo !="ARRDE" :word += f"({row[algo]})"
            word += r"}"

            if j!= (len(algolist)-1) :  word+=" & "
            else : word += r" \\ "

        print(word)
        continue 

    for j, algo in enumerate(algolist) : 
        if j==indexMax : 
            word += r"\color{red} {\textbf{" 
            word += f"{df1[algo][i]:0.2e} $\pm$ {df2[algo][i]:0.2e} "
            if algo !="ARRDE" :word += f"({row[algo]})"
            word += r"}}"

        elif j== indexMin : 
            word += r"\textbf{" 
            word += f"{df1[algo][i]:0.2e} $\pm$ {df2[algo][i]:0.2e} "
            if algo !="ARRDE" :word += f"({row[algo]})"
            word += r"}"
        else : 
            word += f"{df1[algo][i]:0.2e} $\pm$ {df2[algo][i]:0.2e} "  
            if algo !="ARRDE" :word += f"({row[algo]})"

        if j!= (len(algolist)-1) :  word+=" & "
        else : word += r" \\ "

    print(word)
    #print(rf"{probnum} & {df1["LSHADE"][i]:0.2e} $\pm$ {df2["LSHADE"][i]:0.2e} ({row["LSHADE"]}) & {df1["j2020"][i]:0.2e} $\pm$ {df2["j2020"][i]:0.2e} ({row["j2020"]})& {df1["NLSHADE_RSP"][i]:0.2e} $\pm$ {df2["NLSHADE_RSP"][i]:0.2e} ({row["NLSHADE_RSP"]}) & {df1["LSRTDE"][i]:0.2e} $\pm$ {df2["LSRTDE"][i]:0.2e} ({row["LSRTDE"]}) &{df1["ARRDE"][i]:0.2e} $\pm$ {df2["ARRDE"][i]:0.2e}  \\")

    for rn in ["LSHADE", "j2020", "NLSHADE_RSP", "LSRTDE"] : 
        it = row[rn] 
        if it=="-" : nzp[rn][0]+=1
        elif it == "=" : nzp[rn][1]+=1
        elif it =="+" : nzp[rn][2]+=1

print (nzp)


# CEC 2017

In [ ]:
maxevals=100000
filename = resFolder+"results_2017_NLSHADE_RSP_10_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_ARRDE_10_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_j2020_10_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSHADE_10_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSRTDE_10_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17), axis=0), 
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec17), axis=0), 
    })


display(df)
for c in df.columns : print(c, np.mean(df[c]))



print("------------")
df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(30)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(30)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(30)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(30)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
display(df)


In [ ]:
maxevals=30000
filename = resFolder+"results_2017_NLSHADE_RSP_30_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_ARRDE_30_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_j2020_30_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSHADE_30_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSRTDE_30_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17)/goptimum_cec17, axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17)/goptimum_cec17, axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17)/goptimum_cec17, axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17)/goptimum_cec17, axis=0), 
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec17)/goptimum_cec17, axis=0), 
    })

df= df.drop(index=1)
display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")
df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(30)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(30)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(30)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(30)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df

In [ ]:
maxevals=50000
filename = resFolder+"results_2017_NLSHADE_RSP_50_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_ARRDE_50_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_j2020_50_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSHADE_50_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSRTDE_50_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17)/goptimum_cec17, axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17)/goptimum_cec17, axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17)/goptimum_cec17, axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17)/goptimum_cec17, axis=0), 

    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec17)/goptimum_cec17, axis=0), 
    #"JADE" : np.mean(np.abs(data6-goptimum_cec17), axis=0)
    })

df= df.drop(index=1)
display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(30)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(30)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(30)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(30)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df

In [ ]:
maxevals = 100000
filename = resFolder+"results_2017_NLSHADE_RSP_100_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_ARRDE_100_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_j2020_100_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSHADE_100_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2017_LSRTDE_100_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec17)/goptimum_cec17, axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec17)/goptimum_cec17, axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec17)/goptimum_cec17, axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec17)/goptimum_cec17, axis=0), 
    #"jSO" : np.mean(np.abs(data4-goptimum_cec17), axis=0), 
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec17)/goptimum_cec17, axis=0), 
    #"JADE" : np.mean(np.abs(data6-goptimum_cec17), axis=0)
    })

df= df.drop(index=1)
display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(30)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(30)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(30)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(30)], 
    })
for c in df.columns : print(c, np.mean(df[c]))

# CEC 2020

In [ ]:
maxevals=50000
filename = resFolder+"/results_2020_NLSHADE_RSP_5_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_ARRDE_5_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_j2020_5_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSHADE_5_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSRTDE_5_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec20), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec20), axis=0),
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec20), axis=0)
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(10)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(10)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(10)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(10)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df

In [ ]:
maxevals=1000000
filename = resFolder+"results_2020_NLSHADE_RSP_10_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_ARRDE_10_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_j2020_10_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSHADE_10_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSRTDE_10_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec20), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec20), axis=0),
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec20), axis=0)
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(10)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(10)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(10)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(10)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df


In [ ]:
maxevals=3000000
filename = resFolder+"results_2020_NLSHADE_RSP_15_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_ARRDE_15_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_j2020_15_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSHADE_15_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSRTDE_15_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')


df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec20), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec20), axis=0),
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec20), axis=0)
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(10)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(10)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(10)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(10)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df

In [ ]:
maxevals=10000000
filename = resFolder+"results_2020_NLSHADE_RSP_20_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_ARRDE_20_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_j2020_20_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSHADE_20_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2020_LSRTDE_20_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec20), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec20), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec20), axis=0), 
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec20), axis=0),
     "LSRTDE" : np.mean(np.abs(data5-goptimum_cec20), axis=0)
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))

print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(10)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(10)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(10)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(10)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df

# CEC 2022


## 10D

In [ ]:
maxevals=200000
filename = resFolder+"results_2022_NLSHADE_RSP_10_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_ARRDE_10_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_j2020_10_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_LSHADE_10_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_LSRTDE_10_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')
df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec22), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec22), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec22), axis=0),
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec22), axis=0),
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec22), axis=0),
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))
print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(12)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(12)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(12)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(12)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df

## 20D


In [ ]:
maxevals=1000000
filename = resFolder+"results_2022_NLSHADE_RSP_20_"+str(maxevals)+".txt"
data = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_ARRDE_20_"+str(maxevals)+".txt"
data1 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_j2020_20_"+str(maxevals)+".txt"
data2 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_LSHADE_20_"+str(maxevals)+".txt"
data3 = np.loadtxt(filename, delimiter='\t')
filename = resFolder+"results_2022_LSRTDE_20_"+str(maxevals)+".txt"
data5 = np.loadtxt(filename, delimiter='\t')

df = pd.DataFrame({
    "NLSHADE_RSP" : np.mean(np.abs(data-goptimum_cec22), axis=0), 
    "ARRDE" : np.mean(np.abs(data1-goptimum_cec22), axis=0), 
    "j2020" : np.mean(np.abs(data2-goptimum_cec22), axis=0),
    "LSHADE" : np.mean(np.abs(data3-goptimum_cec22), axis=0),
    "LSRTDE" : np.mean(np.abs(data5-goptimum_cec22), axis=0),
    })

display(df)
for c in df.columns : print(c, np.mean(df[c]))
print("------------")

df = pd.DataFrame({
    "NLSHADE_RSP" : [MWUT(data1[:, i], data[:,  i]) for i in range(12)], 
    "j2020" : [MWUT(data1[:, i], data2[:,  i]) for i in range(12)], 
    "LSHADE" : [MWUT(data1[:, i], data3[:,  i]) for i in range(12)], 
    "LSRTDE" : [MWUT(data1[:, i], data5[:,  i]) for i in range(12)], 
    })
for c in df.columns : print(c, np.mean(df[c]))
df

In [ ]:
# 